In [29]:
import numpy as np
import pandas as pd

import glob
import os
from collections import defaultdict

from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input as vgg_preprocess_input

import matplotlib.pyplot as plt
%matplotlib inline

from helpers import *

In [26]:
# Load a model
model = VGG16(weights='imagenet', include_top=False)

LAYERS = ['block4_conv1', 'block4_conv2', 'block4_conv3' , 
          'block5_conv1', 'block5_conv2', 'block5_conv3']  # layer of interest
CAT_FLD = 'images/'        # folder with images separated by category into subfolders
ACT_FLD = 'activations/'

In [3]:
def vgg_image(path, target_size=(224, 224)):
    """Tranform image for vgg with keras module"""
    img = image.load_img(path, target_size=target_size)
    x = image.img_to_array(img)
    x = np.expand_dims(img, axis=0).astype(np.float)
    x = vgg_preprocess_input(x)
    return x


def vgg_images_from_filenames(filenames):
    """Load and preprocess images from filenames"""
    images = [vgg_image(f) for f in filenames]
    images = [x for x in images if x is not None]
    images = np.squeeze(np.vstack(images))
    return images


def get_activations(model, images, layers_names):
    """Get activations for the input from specified layer"""
    
    inp = model.input                                          
    names_outputs = [(l.name, l.output) for l in model.layers if l.name in layers_names] 
    names = [x[0] for x in names_outputs]
    outputs = [x[1] for x in names_outputs]
    functor = K.function([inp] + [K.learning_phase()], outputs) 
    
    return dict(zip(names, functor([images])))

def get_activations_batches(model, filenames, layers, output_fld, img_loader_func, 
                            batch_size=128, crop_model_=True, **kwargs): 
    """Split input into batches and get activations"""
    
    file_gen = batch_generator(filenames, batch_size, equal_size=False)
    
    for l in layers:
        l_fld = os.path.join(output_fld, l)
        if not os.path.exists(l_fld):
            os.makedirs(l_fld)
    
    # get activations
    for batch_n, batch_files in enumerate(file_gen):
        print("batch", batch_n)
        images = img_loader_func(batch_files,  **kwargs)
        activations = get_activations(model, images, layers)
        
        for l_name, v in activations.items():
            for path, l_acts in zip(batch_files, v):
                name = os.path.basename(path).split('.')[0]            
                new_path = os.path.join(output_fld, l_name, name + '.npy')
                np.save(new_path, l_acts)

    print("All activations saved to folder", output_fld)

# Get activations for all images 

In [4]:
for cat in os.listdir(CAT_FLD):

    cat_files = glob.glob(os.path.join(CAT_FLD, cat, '*'))
    print('In category {} found {} files'.format(cat.upper(), len(cat_files)))
    print('Getting activations for files...')
    
    output_cat_fld = os.path.join(ACT_FLD, cat)
    get_activations_batches(model, cat_files, LAYERS, output_fld=output_cat_fld, 
                            img_loader_func=vgg_images_from_filenames, batch_size=128)
    print()

In category FACE found 819 files
Getting activations for files...
batch 0
batch 1
batch 2
batch 3
batch 4
batch 5
batch 6
All activations saved to folder activations_takayuki/face

In category NONFACE found 731 files
Getting activations for files...
batch 0
batch 1
batch 2
batch 3
batch 4
batch 5
All activations saved to folder activations_takayuki/nonface



# For each filter in a layer plot category_selectivity tuning curve

In [23]:
def get_cat_mask(categories, cat):
    return np.array([x == cat for x in categories])
    
def category_selectivity_index(activations, cat_mask, axis=0):
    
    act_norm = norm_values(activations)
    cat = act_norm[cat_mask]    
    noncat = act_norm[~cat_mask]
    selectivity = np.sum(cat, axis)/cat.shape[axis] - np.sum(noncat, axis)/noncat.shape[axis]
    return selectivity

def cat_selective_func(func, activations, labels, **kwargs):
    
    cat_selectivity = []
    
    for l in np.unique(labels):
        category_mask = get_cat_mask(labels, l)
        selectivity = func(activations, category_mask, **kwargs)
        cat_selectivity.append([selectivity])
    cat_selectivity = np.vstack(cat_selectivity)    
    return cat_selectivity

In [5]:
# Load all activations

def load_act_from_fld(fld, layer=''):
    
    label_dict = {}
    activations = []
    labels = []
    
    for (n, cat) in enumerate(os.listdir(fld)):
        label_dict[n] = cat
        cat_fld = os.path.join(fld, cat, layer)
        np_files = glob.glob(cat_fld + '/*.np[yz]')
        labels.extend([n] * len(np_files))
        cat_activations = []
        print('Loading {} files from {}'.format(len(np_files), cat_fld))
        for (i, f) in enumerate(np_files):
            activations.append(np.load(f))

    return np.asarray(activations), labels, label_dict

In [30]:
ACTIVATIONS = defaultdict(lambda : defaultdict())

for l in LAYERS:
    acts, labels, label_dict = load_act_from_fld(ACT_FLD, layer=l)
    ACTIVATIONS[l]['activations'] = acts
    ACTIVATIONS[l]['label_dict'] = label_dict
    ACTIVATIONS[l]['labels'] = labels

Loading 301 files from activations/face/block4_conv1
Loading 301 files from activations/dog/block4_conv1
Loading 301 files from activations/cat/block4_conv1
Loading 301 files from activations/car/block4_conv1
Loading 301 files from activations/face/block4_conv2
Loading 301 files from activations/dog/block4_conv2
Loading 301 files from activations/cat/block4_conv2
Loading 301 files from activations/car/block4_conv2
Loading 301 files from activations/face/block4_conv3
Loading 301 files from activations/dog/block4_conv3
Loading 301 files from activations/cat/block4_conv3
Loading 301 files from activations/car/block4_conv3
Loading 301 files from activations/face/block5_conv1
Loading 301 files from activations/dog/block5_conv1
Loading 301 files from activations/cat/block5_conv1
Loading 301 files from activations/car/block5_conv1
Loading 301 files from activations/face/block5_conv2
Loading 301 files from activations/dog/block5_conv2
Loading 301 files from activations/cat/block5_conv2
Loading

In [31]:
# Find selectivity for filters in different layers

for l, v in ACTIVATIONS.items():  
    print(l)
    cat_sel = cat_selective_func(category_selectivity_index, v['activations'], v['labels'])
    # select by mean filter activation
    cat_sel_filter = np.mean(cat_sel, axis=(1, 2))

    # # select by activation of filter's cantral unit
    # central_unit = np.ceil(cat_sel.shape[1]/2).astype(int)
    # cat_sel_filter = cat_sel[:, central_unit, central_unit, :]
    ACTIVATIONS[l]['filter_selectivity'] = cat_sel_filter

block5_conv3
block5_conv1
block4_conv2
block5_conv2
block4_conv3
block4_conv1


In [32]:
# Convert selectivity index into dataframe

for l, v in ACTIVATIONS.items():  
    print(l)
    
    df = pd.DataFrame(v['filter_selectivity'].T)
    df['filter'] = df.index
    df = df.melt(id_vars=df.columns[-1], value_vars=df.columns[:-1], 
                 var_name='category', value_name='selectivity')

    df['label'] = df['category'].map(v['label_dict'])
    df['layer'] = l
    
    df_path = os.path.join(ACT_FLD, 'selectivity')
    if not os.path.exists(df_path):
        os.makedirs(df_path)
    df.to_csv(os.path.join(df_path, '{}.csv'.format(l)), index=False)
    df.head()

block5_conv3
block5_conv1
block4_conv2
block5_conv2
block4_conv3
block4_conv1


In [28]:
import mpld3 #pip install mpld3
mpld3.enable_notebook()

In [29]:
# Define some CSS to control our custom labels
css = """
table{
  border-collapse: collapse;
}
th{
  color: #ffffff;
  background-color: #c1c1c1;
}
td{
  background-color: #e8e8e8;
}
table, th, td{
  font-family:Arial, Helvetica, sans-serif;
  text-align: right;
  color: 484848;
}
"""
labels = []
for i in range(512):
    label = df.loc[df['filter'] == i, ['label', 'selectivity']]
    label.set_index('label', inplace=True)
    label.columns = ['_Filter {} SI_'.format(i)]
    # .to_html() is unicode; so make leading 'u' go away with str()
    labels.append(str(label.to_html()))

In [31]:
cat_names = df['label'].unique()

ncol = 2
nrow = np.ceil(len(cat_names) / ncol).astype(int)

fig, axs = plt.subplots(nrows=nrow, ncols=nrow, figsize=(12, 12), sharey=True)
axs = np.array(axs)

for n, ax in enumerate(axs.reshape(-1)):
    name = cat_names[n]
    group = df.loc[df['label'] == name, :]
    p = ax.plot(group['filter'], group['selectivity'], 'o', color='orange', mec='darkorange', ms=7, alpha=.7)
    ax.set_title(name)
    ax.set_xlabel('filter')
    ax.set_ylabel('selectivity')
    
    tooltip = mpld3.plugins.PointHTMLTooltip(p[0], labels,
                                       voffset=10, hoffset=10, css=css)
    mpld3.plugins.connect(fig, tooltip)

mpld3.display()